In [9]:
# import base64
# import io

# captcha = "/9j/4AAQSkZJRgABAgAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABQAMgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3miiigQUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRTBNG0zQiRDKih2QMNwU5AJHodrY+h9KAH0UUUAFFQ2t3bX1slzaXEVxA+dssLh1bBwcEcHkEVNQAUUUUAFFMmmjt4ZJppEjijUu7uwCqo5JJPQCq1jq2m6p5n9n6haXfl43/Z5lk25zjOCcZwfypXWwrq9i5RUMl3bQ3MNtLcRJPPu8mJnAaTaMttHU4HJx0qamMKKKKACiiigAooooAKKKKACiiigAooooAKKKrahqFppVhNfX06QW0K7pJH6Af1J6ADkk4FGwN21ZQ8S+IIfDelfa5IJbmaSQQW1tCpLTStnagwDjODz7cZOAfOPh9b6rbfFXVo9cm83Uzp/mTtu3YLGFguRxwCBgcDGBxitLw3rel67r0virXNctLcwySQ6VYzXUcXkRHgyMu8newyDn0zyNuK2k63pMfxp12/fVLJbOWxVI7hrhBG7Yh4DZwTwfyNckpKUoyvpc4pzU5RnfS/9M6/xHpmi6n4k8ODUbu4iv4ZpJrGKL7srJsdtx2ngbR3HU1Nr3g7TvEl/BPqk97Lbwrj7Cs5WBmG7DlRzu+Y8gjoB0qj4i/5KN4L/wC37/0SK6+tlFSck11/RG6jGTkmuv6I5D4Xf8k50r/tt/6OepZPiR4RitoZ21qIpNu2hY3Zhg4O5QuV9twGe2ai+F3/ACTnSv8Att/6Oeqfwo0qwt/BVlqMVpEt7ceb5xtxty7ASEYz1Awi8DjjPWog5csYx7f5EQlPlhGPb/I6O48U6Ha6DFrc2oxLp0uPLmGTuJ7BQMk8HIxkYOcYNGieKdD8RbxpWoxXDpndHyjgDHO1gDjkc4xXA6HounR/GnVLBbVBZ2MP2y1t8ny4ZmEOXVegPP4YGMbRje8aWMFj4h8M65aIkN++rRWksiRrmWORSrbzjJIVcA54BPthqpO3N20BVZ2cnaydg8UQSeJfGmneFpXdNKS1OoXyJKVNwofaicDOAwBPIyDngqDVbxlpFt4Uit/Fmg20VlNYSIl1Bb4iS5gZgChUKRnJHOOOvJVcaX/NY/wDuX/8A24o+KP8AyTnVf+2P/o5KUknGUuv+QpxTjOXVfoHiL/ko3gv/ALfv/RIrr64GaK7h8T/DqK/Lm8S1uVnLvvYyC3UNlsnJznnNd9WlN3cvX9Ea0ndyfn+iCiiitDUKKKKACiiigAooooAKKKKACiiigApk0MdxDJDNGkkUilHR1BVlPBBB6g0+igDH/wCET8Of9C/pX/gHH/hXIad4KiX4o6vPcaDaf2I1mn2cNBGYd5CD5V6A5STPHGefvDPo9FZypxdvIzlShK2mxyuu2lzN498JXMVvK8EH2zzpVQlY90QC7j0GTwM9a6qiiqUbNvuVGNm33OV+HFpc2PgLTLa7t5bedPN3RTIUZcyuRkHkcEGj4cWlzY+AtMtru3lt5083dFMhRlzK5GQeRwQa6qilGFreSsTGmo28lb8v8jhNI0q/T4x6/qj2kqWLWccSTsuFdisXCk/e+42cdMc9RWj42tLm7/4Rz7NbyzeVrltLJ5aFtiDdljjoB611VFL2as13F7Jcrj3dzkPFul6lbarZeKtEh+0XtjG0VzZ7mBuoDztGDjKkkgYOTg8lQDm6g+pfEG8tdM/sjUNL0KCRJ759QjaF7jBOIlUHkHqTnjg8EDd6DRQ6d3vowlSTb10e6OQ8a21/b6loPiGwtZbw6ZcOs9tDFvdopQFdlAIJIA4H+1k8A10el6h/ammxXn2S7tPMz+4u4/LlXBI+ZcnGcZHsRVyiqUbSb7lKFpN33CiiiqLCiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooA//2Q=="

# image_binary = base64.b64decode(captcha)
# image_file = io.BytesIO(image_binary)
# img = Image.open(image_file)
# img.save("captcha.png")

# print("Image saved as captcha.png")


import base64
import json

# Your JSON response
response = '{"status":"Success","statusCode":200,"message":"Captcha Generated","captcha":"/9j/4AAQSkZJRgABAgAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABQAMgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3miiigQUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRUNvd2135v2a4im8qQxSeW4bY46qcdCPSpqACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKAPPvDOv6V4ftvE0+q30VsjeILvaGOWflAdqjJbGRnAOM11OieKdD8RbxpWoxXDpndHyjgDHO1gDjkc4xXM+AtHsm1vxRrDwpJeDWriGN3VSYlUk/IcZBPmEHnkAUfErT106wg8WabBbw6np11HLJPyrSxnCFG243A5Qcn7uQDzg88ZSjDm6HLCU40+bojd8WWmkXsOlRaxdXFuv9pQm2MDMrPP8AMEXcoJUHJ5BGPUVq6pqlloumy6hqE3k2sWN8m0tjJCjgAnqRXOePv+ZX/wCxgtP/AGaj4o/8k51X/tj/AOjkq5S5eZroXKXLztLZfoWJfiH4SivxZNrduZSyruQM0eTjH7wDZjnk5wO+MVc1zxdoPhyaKHVtQS3llUuiBGdtvTJCg4Hpnrg46Gpk8NaLHokmjJptuunyrtkhVcB+ANxPUtwPmznIBzmuH+DNrDLod7qkqeZfG4+y+e5LMIkjj2oCegHoPQegwnKopKOmonOopKLtr/X9bHoOmarYazZrd6ddxXMDY+eNs4OAcEdQcEcHBGaxNN0zRf8AhP8AWdUtbu4k1ZYY4byBuI41ZVK4+UZJEY/iPes7SbGDRPitfWOnIlvZ3ukpdyW8caqiyJJ5Y2gAYGMnHcsT6Ys+Hf8Ako3jT/tx/wDRJo5r2uuv6MfNzcvMtb/ozoLLWtO1G/vrGzukmuLFlS5VAcRsc4GehPBBwTgjBwaNR1rTtKms4b66SGW9mEFuhBLSOeMADtyMnoMjJ5rlvGlhJol/D420tbdbiyUpfxPIYhdwnCgFs4LDjGQcnb1KqpZ4RT/hLdVbxjd3nmpFJLBptmhwLROVJkAJ/eMuCeTwQeflCntJX5Ov6B7SXNydf07/ANdTtbq7trG2e5u7iK3gTG6WZwirk4GSeByQK5u1+JHhG8uUgi1qJXfODNG8S8DPLMoA6dzVHXbRfEPxJ0zRr7Y+m2Vi2otbsCVnkL+WAwzggcEcH+IHIaupvdE0vUdMGm3dhby2artSEoAsYAKjbj7pAJAIwR2p3m2+XoPmnJvltoX6K5D4cyXKaDeaXcz+f/ZOoTWEcxBBdExgnJOPvYA7AAdq6+rjLmSZpCXNFSCiiiqKCiiigAooooAKKKKACiiigAooooA5XwTaXNp/wkf2m3lh83XLmWPzEK70O3DDPUH1o+I9pc33gLU7a0t5bid/K2xQoXZsSoTgDk8AmuqqG7kmhs55baD7ROkbNHDvCeYwHC7jwMnjPao5FyOJm6a9m4HLePv+ZX/7GC0/9mqX4j2lzfeAtTtrS3luJ38rbFChdmxKhOAOTwCaow6V4j8T+IdP1DxBZW+mabpsxmhsUuDLJJKFXazsp2kBskHr1BBBzXcVCXPzeZCjz819EwrhPhLpV/pPhGaPULSW1klvHlSOZdrbdqLkqeRyp6/XoRXd0Vo43kpdjRwTkpdjlfslz/wtf7Z9nl+y/wBh+V5+w7N/n527umcc4o0K0uYfHvi25lt5Ugn+x+TKyELJtiIbaehweDjpXVUUuTW/ncXs1e/nc4e60298ZeLUOpae9v4e0eZwsNyGBvpsY37c4KDqCc5BI53EK+TTr/wv4zhu9Lju7nR9ZuG+32yJ5gt526TDnKgn7x6AA/7AHa0UvZrfqL2S3673PO9ei1Q/FmC50go9za6L5xtncotyvnMpjLZ4J3ZBOQGVcjuLj+ONau7WOHS/BmsDUpFxi9i8mCNtpJO843AEDg7c+oPFan9l3v8Awsr+1/J/0H+x/s3m7h/rPO3bcZz05zjFdJUxhK7s7akRpyu7O2pg+EPD7+G9BW0nn+0XssjXF3PuY+bK3U5Y5PAAzxnGcAk1vUUVrFKKsjeMVFWQUUUUxhRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQB//9k="}'

# Parse the JSON to get the base64 string
data = json.loads(response)
base64_image = data['captcha']

# Decode the base64 string
image_data = base64.b64decode(base64_image)

# Write the binary data to a file
with open('captcha_image.jpg', 'wb') as file:
    file.write(image_data)

print("Image saved as captcha_image.jpg")




Image saved as captcha_image.jpg


In [10]:
import pytesseract 
import cv2

img = cv2.imread('captcha_image.jpg') 
text = pytesseract.image_to_string(img)

print(text.lower())

hjmixp



In [22]:
import pytesseract 
import requests
import base64
import json
import cv2


In [46]:

def get_captcha_data():
    captcha_data = None

    url = 'https://gateway-voters.eci.gov.in/api/v1/captcha-service/generateCaptcha' 

    headers = {
    'Accept': 'application/json, text/plain, */*',
    'Origin': 'https://electoralsearch.eci.gov.in',
    'appName': 'ELECTORAL_SEARCH'
    }
    
    try:
        response = requests.get(url, headers=headers)
    

        data = json.loads(response.text)

        if data['status'] == 'Success' and data['statusCode'] == 200:
            captcha_data = {
                'captcha': data['captcha'],
                'id': data['id']
            }
            # print(captcha_data)
        else:
            print("Invalid response")
    except Exception as e:
        print(str(e))
        return None    
    return captcha_data    

In [28]:
get_soup()

{'captcha': '/9j/4AAQSkZJRgABAgAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABQAMgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3miiigQUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRVa91Cy02ETX95b2sTNsDzyqiluTjJPXg/lU0U0dxCk0MiSRSKHR0YFWU8ggjqDRcLrYfRUN1d

In [47]:
def get_code(base64_image):
    # base64_image = data['captcha']

    # Decode the base64 string
    image_data = base64.b64decode(base64_image)

    # Write the binary data to a file
    with open('captcha_image.jpg', 'wb') as file:
        file.write(image_data)

    print("Image saved as captcha_image.jpg")

    img = cv2.imread('captcha_image.jpg') 
    text = pytesseract.image_to_string(img)

    return text.lower()


In [30]:
captcha_data = get_soup()
print(captcha_data)

{'captcha': '/9j/4AAQSkZJRgABAgAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABQAMgDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD3miiigQUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUUUUAFFFFABRRRQAUVDdXdtY2z3N3cRW8CY3SzOEVcnAyTwOSBWb/wlnhz/oYNK/8AAyP/ABpOSW7E5Jbs2KK

In [48]:
captcha_data = get_soup()

captcha = captcha_data['captcha']
id = captcha_data['id']

code = get_code(captcha)

print(id, code )






Image saved as captcha_image.jpg
87F716BE5C02CB6A64F0D68F476B29B1 m8kmaa



In [ ]:
curl 'https://gateway.eci.gov.in/api/v1/elastic/search-by-epic-from-national-display' \
  -H 'Accept: application/json, text/plain, */*' \
  -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/json' \
  -H 'DNT: 1' \
  -H 'Origin: https://electoralsearch.eci.gov.in' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-site' \
  -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36' \
  -H 'applicationName: ELECTORAL_SEARCH' \
  -H 'sec-ch-ua: "Chromium";v="119", "Not?A_Brand";v="24"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  --data-raw '{"isPortal":true,"epicNumber":"ZRS2855062","captchaId":"87F716BE5C02CB6A64F0D68F476B29B1","captchaData":"m8kmaa","securityKey":"na"}' \
  --compressed

In [114]:
import requests
import json

def get_voter_info(epicno, captcha_id, captcha_data):
    url = 'https://gateway.eci.gov.in/api/v1/elastic/search-by-epic-from-national-display'

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        'DNT': '1',
        'Origin': 'https://electoralsearch.eci.gov.in',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'applicationName': 'ELECTORAL_SEARCH',
        'sec-ch-ua': '"Chromium";v="119", "Not?A_Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"'
    }

    data = {
        "isPortal": True,
        "epicNumber": epicno,
        "captchaId": captcha_id,
        "captchaData": captcha_data,
        "securityKey": "na"
    }

    print(data)

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to get data. Status code: {response.text }")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# epicno = "ZRS2855062" 
# captcha_id = "967A07FFA18E6A3ACF254FBA8E6F631A"
# captcha_data = "hjmixp"

# data = get_voter_info(epicno, captcha_id, captcha_data)
# print(data)

In [105]:
# import subprocess
# import json

# def get_voter_info(epicno, captcha_id, captcha_data):
#     command = "curl 'https://gateway.eci.gov.in/api/v1/elastic/search-by-epic-from-national-display' " \
#           "-H 'Accept: application/json, text/plain, */*' " \
#           "-H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7' " \
#           "-H 'Connection: keep-alive' " \
#           "-H 'Content-Type: application/json' " \
#           "-H 'DNT: 1' " \
#           "-H 'Origin: https://electoralsearch.eci.gov.in' " \
#           "-H 'Sec-Fetch-Dest: empty' " \
#           "-H 'Sec-Fetch-Mode: cors' " \
#           "-H 'Sec-Fetch-Site: same-site' " \
#           "-H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36' " \
#           "-H 'applicationName: ELECTORAL_SEARCH' " \
#           "-H 'sec-ch-ua: \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"' " \
#           "-H 'sec-ch-ua-mobile: ?0' " \
#           "-H 'sec-ch-ua-platform: \"Linux\"' " \
#           "--data-raw '{\"isPortal\":true,\"epicNumber\":\""+"hjhjkhkj"+"\",\"captchaId\":\""+"jhgjhjkhjhk"+"\",\"captchaData\":\""+"kkjhjhjk"+"\",\"securityKey\":\"na\"}' " \
#           "--compressed"
#     print(command.split())


#     # try:
#     #     result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#     #     return result.stdout
#     # except subprocess.CalledProcessError as e:
#     #     print(f"Error: {e.returncode}, {e.stderr}")
#     #     return None

# # Example usage
# # epicno = "Your_Epic_Number"
# # captcha_id = "Your_Captcha_Id"
# # captcha_data = "Your_Captcha_Data"

# # response = perform_curl_request(epicno, captcha_id, captcha_data)
# # print(response)


In [84]:
# import subprocess

# def perform_curl_request_as_string(command):
#     try:
#         result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
#         return result.stdout
#     except subprocess.CalledProcessError as e:
#         return f"Error: {e.returncode}, {e.stderr}"

# # Example usage
# command = "curl 'https://gateway.eci.gov.in/api/v1/elastic/search-by-epic-from-national-display' -H 'Accept: application/json, text/plain, */*' -H 'Accept-Language: en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7' -H 'Connection: keep-alive' -H 'Content-Type: application/json' -H 'DNT: 1' -H 'Origin: https://electoralsearch.eci.gov.in' -H 'Sec-Fetch-Dest: empty' -H 'Sec-Fetch-Mode: cors' -H 'Sec-Fetch-Site: same-site' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36' -H 'applicationName: ELECTORAL_SEARCH' -H 'sec-ch-ua: \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"' -H 'sec-ch-ua-mobile: ?0' -H 'sec-ch-ua-platform: \"Linux\"' --data-raw '{\"isPortal\":true,\"epicNumber\":\"ZRS2855062\",\"captchaId\":\"142CE46F02549624447ECAEC86D292E1\",\"captchaData\":\"nacrfb\",\"securityKey\":\"na\"}' --compressed"

# response = perform_curl_request_as_string(command)
# print(response)

[{"index":"national-electoral-display","id":"68375803_ZRS2855062_S25","score":17.278444,"sortValues":[],"content":{"id":"68375803_ZRS2855062_S25","epicId":68375803,"epicNumber":"ZRS2855062","formReferenceNo":null,"applicantFirstName":"ADITYA","applicantFirstNameL1":"আদিত্য","applicantFirstNameL2":null,"applicantLastName":"PASWAN","applicantLastNameL1":"পাসওয়ান","applicantLastNameL2":null,"relationName":"BIRENDRA","relationNameL1":"বীরেন্দ্র","relationNameL2":null,"age":20,"gender":"M","partNumber":40,"partId":19083,"partName":"Kedarnath Hindi Primary School ","partNameL1":"কেদারনাথ হিন্দী প্রাইমারী স্কুল ","partSerialNumber":437,"asmblyName":"Matigara-Naxalbari","asmblyNameL1":"মাটিগাড়া - নক্সালবাড়ি","acId":26,"acNumber":25,"prlmntName":"DARJEELING","prlmntNameL1":"দার্জিলিং","prlmntNo":"4","districtValue":"DARJEELING","districtValueL1":"দার্জিলিং","districtCd":"S2503","districtId":3,"districtNo":3,"stateName":"West Bengal","stateNameL1":"पश्चिम बंगाल","stateId":24,"stateCd":"S25","isA

In [124]:
def get_user_info():
    got_data = False
    while got_data == False:
        captcha_data = get_captcha_data()

        captcha = captcha_data['captcha']
        captcha_id = captcha_data['id']

        captcha_data = get_code(captcha).split("\n")[0]

        print(captcha_data)

        epicno = "ZRS2855062" 
        # captcha_id = "967A07FFA18E6A3ACF254FBA8E6F631A"
        # captcha_data = "hjmixp"

        print(epicno, captcha_id, captcha_data)

        data = get_voter_info(epicno, captcha_id, captcha_data)
        if data != None:
            got_data = True
            print(data)
        else:
            print("wrong captcha")    

In [126]:
get_user_info()

Image saved as captcha_image.jpg
mvdonz
ZRS2855062 E6F3E6E5062DA2E09BE428FE2A213798 mvdonz
{'isPortal': True, 'epicNumber': 'ZRS2855062', 'captchaId': 'E6F3E6E5062DA2E09BE428FE2A213798', 'captchaData': 'mvdonz', 'securityKey': 'na'}
Failed to get data. Status code: []
wrong captcha
Image saved as captcha_image.jpg
vhlg97
ZRS2855062 6962A381743C9861F8D51542752A72F4 vhlg97
{'isPortal': True, 'epicNumber': 'ZRS2855062', 'captchaId': '6962A381743C9861F8D51542752A72F4', 'captchaData': 'vhlg97', 'securityKey': 'na'}
Failed to get data. Status code: []
wrong captcha
Image saved as captcha_image.jpg
yqs5771
ZRS2855062 8E480C254EA99CD366CBA68495538FC2 yqs5771
{'isPortal': True, 'epicNumber': 'ZRS2855062', 'captchaId': '8E480C254EA99CD366CBA68495538FC2', 'captchaData': 'yqs5771', 'securityKey': 'na'}
Failed to get data. Status code: []
wrong captcha
Image saved as captcha_image.jpg
tv9z6d
ZRS2855062 5809A8499DB0672F246C73492DCEFCEB tv9z6d
{'isPortal': True, 'epicNumber': 'ZRS2855062', 'captchaId

Location not found
